# Upload CSV files in to postgres using Python.

In [1]:
!pip install pyarrow

In [3]:
!pip install fastparquet

In [4]:
import psycopg2
import pandas as pd

In [76]:
def create_database():
    # The below syntax is used for connecting to database present in the sql server.
    conn=psycopg2.connect(host='localhost', user='postgres',dbname='postgres', password='password')
    conn.set_session(autocommit=True)
    # creating cursor used to apply quries on database
    cur=conn.cursor()
    
    
    #create sparkify database with UTF encoding
    cur.execute("drop database sales_revenue")
    cur.execute("create database sales_revenue")
    
    #close connection to default database
    conn.close()
    
    # connect to the sparkify database
    conn=psycopg2.connect(host='localhost', user='postgres',dbname='sales_revenue', password='password')
    cur=conn.cursor()
    
    return cur,conn
    

In [77]:
def drop_table(conn,curr):
    for query in drop_table_quries:
        curr.execute(query)
        conn.commit()

In [78]:
def create_table(conn,curr):
    for query in create_table_quries:
        curr.execute(query)
        conn.commit()

# There are mainly three csv files avaialble in this jupyter notebook.

1) sales form different states

2) revenue from different states

3) states with region codes

In [79]:
sales=pd.read_csv("Sales_revenue_region/Sales.csv")

In [80]:
sales.head()

,Date,State Id,Region,Type,Sales,Queue
0,1/1/2019,1,Central,Residential,20.0,Store
1,1/2/2019,1,Central,Residential,342.0,Store
2,1/3/2019,1,Central,Residential,187.0,Store
3,1/4/2019,1,Central,Residential,215.0,Store
4,1/5/2019,1,Central,Residential,46.0,Store


In [81]:
revenue=pd.read_csv("Sales_revenue_region/revenue.csv")

In [82]:
revenue.head()

,Date,State Id,Region,Type,Revenue,Queue
0,1/1/2019,1,Central,Residential,10056.0,Store
1,1/2/2019,1,Central,Residential,5263.0,Store
2,1/3/2019,1,Central,Residential,216.0,Store
3,1/4/2019,1,Central,Residential,0.0,Store
4,1/5/2019,1,Central,Residential,23037.0,Store


In [83]:
state_code=pd.read_csv("Sales_revenue_region/Region_state_code.csv")

In [84]:
state_code.head()

,Region,State,State Id
0,Central,Jharkhand,1
1,Central,Odisha,2
2,Central,Uttar Pradesh,3
3,West,Goa,4
4,West,Gujarat,5


>Here state id is giving details about region,so we can delete that column in sales and revenue files.


In [85]:
sales=sales.drop('Region',axis=1)
revenue=revenue.drop('Region',axis=1)

# Create a datamodel using psql.

Idea is to create datamodel in sql database and `not a simple bussiness model.

In [86]:
cur,conn=create_database()

Creating a sales table in RDBMS!

In [87]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30384 entries, 0 to 30383
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      30384 non-null  object 
 1   State Id  30384 non-null  int64  
 2   Type      30384 non-null  object 
 3   Sales     30054 non-null  float64
 4   Queue     30384 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [88]:
sales_table=("""create table if not exists sales_table(Date varchar,
State_Id int ,
Type varchar,
Sales float,
Queue varchar)""")

In [89]:
cur.execute(sales_table)
conn.commit()

Creating a revenue table in RDBMS!

In [90]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30384 entries, 0 to 30383
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      30384 non-null  object 
 1   State Id  30384 non-null  int64  
 2   Type      30384 non-null  object 
 3   Revenue   30384 non-null  float64
 4   Queue     30384 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [91]:
revenue_table=("""create table if not exists revenue_table(Date varchar,
State_Id int ,
Type varchar,
revenue float,
Queue varchar)""")


In [92]:
cur.execute(revenue_table)
conn.commit()

Creating a state_code table in RDBMS!

In [58]:
state_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Region    12 non-null     object
 1   State     12 non-null     object
 2   State Id  12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


In [93]:
state_codes=("""create table if not exists state_codes(Region varchar,
State varchar,
State_Id int primary key)""")

In [94]:
cur.execute(state_codes)

In [95]:
conn.commit()

Insert data into the above created tables! 

The data is available in the csv files, the one avaialbe in python notebook!

In [96]:
sales_insert=("""insert into sales_table(
Date,
State_Id,
Type,
Sales,
Queue) values(%s,%s,%s,%s,%s)""")

In [97]:
for i,row in sales.iterrows():
    cur.execute(sales_insert,list(row))

In [98]:
revenue_insert=("""insert into revenue_table(
Date,
State_Id ,
Type ,
revenue ,
Queue ) values(%s,%s,%s,%s,%s)""")

In [99]:
for i,row in revenue.iterrows():
    cur.execute(revenue_insert,list(row))

In [100]:
state_codes_insert=("""insert into state_codes(
Region ,
State ,
State_Id  ) values(%s,%s,%s)""")

In [103]:
for i,row in state_code.iterrows():
    cur.execute(state_codes_insert,list(row))

In [104]:
cur.close()
conn.close()